# 🌳 Tree Traversal Algorithms

**Phase 3 (Interview Preparation) - Data Structures Module 1**

**Prerequisites**: Phase 1-2 complete understanding of collections and functions

Master the three fundamental tree traversal algorithms that form the basis of all tree problems!

---

## 🏗️ Tree Node Structure

Understanding binary tree node implementation and common patterns.

In [ ]:
// Binary Tree Node definition
case class TreeNode[T](
  value: T,
  left: Option[TreeNode[T]] = None,
  right: Option[TreeNode[T]] = None
) {
  
  // Tree properties
  def height: Int = {
    val leftHeight = left.map(_.height).getOrElse(0)
    val rightHeight = right.map(_.height).getOrElse(0)
    1 + math.max(leftHeight, rightHeight)
  }
  
  def isBalanced: Boolean = {
    val leftHeight = left.map(_.height).getOrElse(0)
    val rightHeight = right.map(_.height).getOrElse(0)
    math.abs(leftHeight - rightHeight) <= 1
  }
  
  def size: Int = 1 + left.map(_.size).getOrElse(0) + right.map(_.size).getOrElse(0)
}

// Sample tree building
def createSampleTree(): TreeNode[Int] = {
  TreeNode(1,
    left = Some(TreeNode(2,
      left = Some(TreeNode(4)),
      right = Some(TreeNode(5,
        left = Some(TreeNode(8))))
    )),
    right = Some(TreeNode(3,
      left = Some(TreeNode(6)),
      right = Some(TreeNode(7))
    ))
  )
}

// Test tree properties
val sampleTree = createSampleTree()
println(s"Tree height: ${sampleTree.height}")
println(s"Tree balance: ${sampleTree.isBalanced}")
println(s"Tree size: ${sampleTree.size}")
println()

## 🔍 Inorder Traversal (Left → Root → Right)

Most common traversal - produces sorted output for BSTs

In [ ]:
// Inorder Traversal: Recursive approach (O(n) time, O(h) space)
def inorderRecursive[T](root: Option[TreeNode[T]]): List[T] = root match {
  case None => Nil
  case Some(node) =>
    inorderRecursive(node.left) ::: List(node.value) ::: inorderRecursive(node.right)
}

// Inorder Traversal: Iterative approach using stack (O(n) time, O(h) space)
import scala.collection.mutable.Stack

def inorderIterative[T](root: Option[TreeNode[T]]): List[T] = {
  val result = scala.collection.mutable.ListBuffer[T]()
  val stack = Stack[TreeNode[T]]()
  var current = root
  
  while (current.isDefined || stack.nonEmpty) {
    // Go to leftmost node
    while (current.isDefined) {
      stack.push(current.get)
      current = current.get.left
    }
    
    // Process node
    current = Some(stack.pop())
    result += current.get.value
    current = current.get.right
  }
  
  result.toList
}

// Morris Traversal: O(n) time, O(1) space (except result list)
def morrisInorder[T](root: Option[TreeNode[T]]): List[T] = {
  val result = scala.collection.mutable.ListBuffer[T]()
  var current = root
  
  while (current.isDefined) {
    if (current.get.left.isEmpty) {
      // No left subtree, visit current
      result += current.get.value
      current = current.get.right
    } else {
      // Find inorder predecessor
      var predecessor = current.get.left
      while (predecessor.get.right.isDefined && predecessor.get.right.get != current.get) {
        predecessor = predecessor.get.right
      }
      
      if (predecessor.get.right.isEmpty) {
        // Link to current
        predecessor.get.right = current
        current = current.get.left
      } else {
        // Unlink and visit current
        predecessor.get.right = None
        result += current.get.value
        current = current.get.right
      }
    }
  }
  
  result.toList
}

// Test all inorder traversals
val tree = createSampleTree()
println("Inorder Traversal Comparisons:")
println(s"Recursive:     ${inorderRecursive(Some(tree))}")
println(s"Iterative:     ${inorderIterative(Some(tree))}")
println(s"Morris:        ${morrisInorder(Some(tree))}")
println("All should be identical!")
println()

## 🔍 Preorder Traversal (Root → Left → Right)

Used for copying trees and getting prefix notation

In [ ]:
// Preorder Traversal: Recursive (O(n) time, O(h) space)
def preorderRecursive[T](root: Option[TreeNode[T]]): List[T] = root match {
  case None => Nil
  case Some(node) =>
    List(node.value) ::: preorderRecursive(node.left) ::: preorderRecursive(node.right)
}

// Preorder Traversal: Iterative using stack (O(n) time, O(h) space)
def preorderIterative[T](root: Option[TreeNode[T]]): List[T] = {
  val result = scala.collection.mutable.ListBuffer[T]()
  val stack = Stack[TreeNode[T]]()
  
  root.foreach(stack.push)
  
  while (stack.nonEmpty) {
    val node = stack.pop()
    result += node.value
    
    // Push right first, then left (so left is processed first)
    node.right.foreach(stack.push)
    node.left.foreach(stack.push)
  }
  
  result.toList
}

// Test preorder traversals
println("Preorder Traversal Comparisons:")
println(s"Recursive:     ${preorderRecursive(Some(tree))}")
println(s"Iterative:     ${preorderIterative(Some(tree))}")
println()

## 🔍 Postorder Traversal (Left → Right → Root)

Used for deleting/cleaning up tree nodes and postfix notation

In [ ]:
// Postorder Traversal: Recursive (O(n) time, O(h) space) 
def postorderRecursive[T](root: Option[TreeNode[T]]): List[T] = root match {
  case None => Nil
  case Some(node) =>
    postorderRecursive(node.left) ::: postorderRecursive(node.right) ::: List(node.value)
}

// Postorder Traversal: Iterative using two stacks (O(n) time, O(n) space)
def postorderIterativeTwoStacks[T](root: Option[TreeNode[T]]): List[T] = {
  val result = scala.collection.mutable.ListBuffer[T]()
  val stack1 = Stack[TreeNode[T]]()
  val stack2 = Stack[TreeNode[T]]()
  
  root.foreach(stack1.push)
  
  while (stack1.nonEmpty) {
    val node = stack1.pop()
    stack2.push(node)
    
    // Push left then right
    node.left.foreach(stack1.push)
    node.right.foreach(stack1.push)
  }
  
  while (stack2.nonEmpty) {
    result += stack2.pop().value
  }
  
  result.toList
}

// Postorder Traversal: Iterative single stack (O(n) time, O(h) space)
def postorderIterativeSingle[T](root: Option[TreeNode[T]]): List[T] = {
  val result = scala.collection.mutable.ListBuffer[T]()
  val stack = Stack[TreeNode[T]]()
  var current = root
  var lastVisited: Option[TreeNode[T]] = None
  
  while (current.isDefined || stack.nonEmpty) {
    while (current.isDefined) {
      stack.push(current.get)
      current = current.get.left
    }
    
    val peek = stack.head
    if (peek.right.isDefined && lastVisited != peek.right) {
      current = peek.right
    } else {
      result += peek.value
      lastVisited = Some(stack.pop())
    }
  }
  
  result.toList
}

// Test postorder traversals
println("Postorder Traversal Comparisons:")
println(s"Recursive:      ${postorderRecursive(Some(tree))}")
println(s"Iterative(2₂S):   ${postorderIterativeTwoStacks(Some(tree))}")
println(s"Iterative(1S):   ${postorderIterativeSingle(Some(tree))}")
println()

## 📊 Level Order Traversal (Breadth First)

Iterative BFS traversal using queue - visits level by level

In [ ]:
// Level Order (BFS) Traversal using queue
def levelOrder[T](root: Option[TreeNode[T]]): List[List[T]] = {
  import scala.collection.mutable.Queue
  
  val result = scala.collection.mutable.ListBuffer[List[T]]()
  val queue = Queue[TreeNode[T]]()
  
  root.foreach(queue.enqueue)
  
  while (queue.nonEmpty) {
    val levelSize = queue.size
    val level = scala.collection.mutable.ListBuffer[T]()
    
    for (_ <- 0 until levelSize) {
      val node = queue.dequeue()
      level += node.value
      
      node.left.foreach(queue.enqueue)
      node.right.foreach(queue.enqueue)
    }
    
    result += level.toList
  }
  
  result.toList
}

// Variant: Single list level order
def levelOrderFlat[T](root: Option[TreeNode[T]]): List[T] = {
  import scala.collection.mutable.Queue
  
  val result = scala.collection.mutable.ListBuffer[T]()
  val queue = Queue[TreeNode[T]]()
  
  root.foreach(queue.enqueue)
  
  while (queue.nonEmpty) {
    val node = queue.dequeue()
    result += node.value
    
    node.left.foreach(queue.enqueue)
    node.right.foreach(queue.enqueue)
  }
  
  result.toList
}

// Test level order traversal
println("Level Order Traversals:")
println(s"Level by level: ${levelOrder(Some(tree))}")
println(s"Flat: ${levelOrderFlat(Some(tree))}")
println()

## 🧩 Tree Construction from Traversals

In [ ]:
// Build tree from inorder and preorder traversals
def buildTreeFromInorderPreorder(inorder: Array[Int], preorder: Array[Int]): Option[TreeNode[Int]] = {
  import scala.collection.mutable.HashMap
  
  // Create value to index mapping for inorder
  val inorderMap = HashMap[Int, Int]()
  inorder.zipWithIndex.foreach { case (value, index) =>
    inorderMap(value) = index
  }
  
  var preorderIndex = 0
  
  def buildHelper(left: Int, right: Int): Option[TreeNode[Int]] = {
    if (left > right) return None
    
    val rootValue = preorder(preorderIndex)
    preorderIndex += 1
    
    val inorderIndex = inorderMap(rootValue)
    
    val root = TreeNode(rootValue,
      left = buildHelper(left, inorderIndex - 1),
      right = buildHelper(inorderIndex + 1, right)
    )
    
    Some(root)
  }
  
  buildHelper(0, inorder.length - 1)
}

// Test tree construction
val inorderSeq = Array(4, 8, 5, 2, 1, 6, 3, 7)
val preorderSeq = Array(1, 2, 4, 5, 8, 3, 6, 7)

val reconstructedTree = buildTreeFromInorderPreorder(inorderSeq, preorderSeq)
println("Tree Construction:")
println(s"Inorder:   ${inorderSeq.mkString("[", ", ", "]")}")
println(s"Preorder:  ${preorderSeq.mkString("[", ", ", "]")}")
println(s"Reconstructed inorder: ${inorderRecursive(reconstructedTree)}")
println(s"Reconstructed preorder: ${preorderRecursive(reconstructedTree)}")
println("Should match original sequences!")
println()

## 🎯 Interview Tips: Tree Traversal Questions

### **Common Interview Patterns:**
- **Recursive vs Iterative**: Always provide both solutions when space-optimized
- **Time Complexity**: O(n) for all traversals since each node visited once
- **Space Complexity**: 
  - Recursive: O(h) where h = height
  - Iterative: O(h) with stack or O(n) with queue for level order
  - Morris: O(1) extra space (excluding result)

### **Edge Cases to Consider:**
- Empty tree (None)
- Single-node tree
- Left-skewed tree (O(n) space for recursion)
- Right-skewed tree
- Balanced tree
- Null node values

### **Follow-up Questions:**
- Can you do it iteratively?
- What's the space complexity?
- Can you do it with O(1) extra space?
- How would you handle large trees?
- Can you modify the tree during traversal?